In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [12]:
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [3]:
r = requests.get("https://www.heraldmalaysia.com/newscategory/Malaysian-Church-News/5",headers = headers)
soup = BeautifulSoup(r.content, "html.parser")
a = soup.find('div', attrs = {"class":"menu_box"})
links = [x.find('a').get('href') for x in a.find_all('li')]
links = [i for i in links if 'category' in i]
links

['https://www.heraldmalaysia.com/newscategory/Malaysian-Church-News/5',
 'https://www.heraldmalaysia.com/newscategory/news_in_malay/7',
 'https://www.heraldmalaysia.com/newscategory/Global-News/2',
 'https://www.heraldmalaysia.com/newscategory/Editorial/8']

In [13]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "html.parser")

    div = soup.find_all('div', attrs = {"class":"news_lst_right"})
    
    if len(div) == 0:
        return

    for link in soup.find_all('div', attrs = {"class":"news_lst_right"}):

        href = link.find('a').get('href')

        hrefs.append(href)

In [14]:
max_worker = 5


for url in links:
    
    print(url)
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    mok = soup.find('div',attrs={"class":"pagination"})
    a = mok.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    max_page = max([int(a_.split('/')[-1].replace('/','')) for a_ in a])
    print(max_page)

    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()

https://www.heraldmalaysia.com/newscategory/Malaysian-Church-News/5
8650


 22%|████████▋                               | 375/1730 [09:49<37:32,  1.66s/it]

HTTPSConnectionPool(host='www.heraldmalaysia.com', port=443): Max retries exceeded with url: /newscategory/Malaysian-Church-News/5/1876 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x12f866990>, 'Connection to www.heraldmalaysia.com timed out. (connect timeout=None)'))


100%|███████████████████████████████████████| 1730/1730 [46:10<00:00,  1.60s/it]


https://www.heraldmalaysia.com/newscategory/news_in_malay/7
8240


...

 74%|███████████████████████████▎         | 2738/3710 [1:06:32<21:44,  1.34s/it]

HTTPSConnectionPool(host='www.heraldmalaysia.com', port=443): Max retries exceeded with url: /newscategory/Global-News/2/13691 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11ae52750>: Failed to resolve 'www.heraldmalaysia.com' ([Errno 8] nodename nor servname provided, or not known)"))


100%|█████████████████████████████████████| 3710/3710 [1:29:02<00:00,  1.44s/it]


https://www.heraldmalaysia.com/newscategory/Editorial/8
1100


100%|█████████████████████████████████████████| 220/220 [04:00<00:00,  1.09s/it]


In [45]:
hrefs = list(set(hrefs))

In [46]:
len(hrefs)

36562

In [19]:
with open(f'herald-link.json', 'a') as f:
    json.dump(hrefs, f)

In [41]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        content = soup.find('div', class_="content_left")
        headline = content.find('h1').text
        subheadline = soup.find('span').text
        h = soup.find('div', class_="news_details")
        p = [p.text for p in h.find_all('p') if p.text and len(p.text.split()) > 5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x,'headline': headline,'subheadline':subheadline, 'content': p}
    
    return data

In [42]:
process_url(hrefs[0])

{'url': 'https://www.heraldmalaysia.com/news/gereja-malaysia-singapura-tidak-benarkan-sakramen-perkahwinan-secara-online/53403/7',
 'headline': 'Gereja Malaysia, Singapura tidak benarkan Sakramen Perkahwinan secara online ',
 'subheadline': 'Gereja Katolik Romawi tidak akan menjalankan perkahwinan dalam talian sepanjang Misa ditangguhkan bagi membendung penyebaran covid-19.\n',
 'content': ["\nMay 16, 2020\n\nRelated News\n\nPengampunan putuskan lingkaran kebencianSidang Sinode berdoa keamanan untuk Timur TengahBumi tidak dapat menampung kita lagiPerintah Tuhan sarana tepat meraih kebahagiaanLektor, pemazmur sebagai duta TuhanHati kita dicipta untuk mengasihi\n\n\n\n\nKUCHING: Gereja Katolik Romawi tidak akan menjalankan perkahwinan dalam talian sepanjang Misa ditangguhkan bagi membendung penyebaran covid-19. Uskup Agung Poh mengatakan Katolik dilarang untuk melakukan perkahwinan dalam talian kerana lafaz sumpah dan berkat dalam talian adalah tidak sah. “Bagi Katolik, upacara perkahwin

In [43]:
max_workers = 3

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('heraldmalaysia.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

  5%|█▉                                  | 1970/36562 [10:50<2:53:12,  3.33it/s]

HTTPSConnectionPool(host='www.heraldmalaysia.com', port=443): Max retries exceeded with url: /news/more-than-200-catholics-pray-and-march-in-silence-over-looted-church/40912/2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10d42f6d0>, 'Connection to www.heraldmalaysia.com timed out. (connect timeout=None)'))


100%|███████████████████████████████████| 36562/36562 [3:25:37<00:00,  2.96it/s]
